In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,r2_score,confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [8]:
df_1 = pd.read_csv('train_motion_data.csv')
df_2 = pd.read_csv('test_motion_data.csv')

data = pd.concat([df_1, df_2])
data


,AccX,AccY,AccZ,GyroX,GyroY,GyroZ,Class,Timestamp
0,0.000000,0.000000,0.000000,0.059407,-0.174707,0.101938,NORMAL,3581629
1,-1.624864,-1.082492,-0.204183,-0.028558,0.051313,0.135536,NORMAL,3581630
2,-0.594660,-0.122410,0.220502,-0.019395,-0.029322,0.087888,NORMAL,3581630
3,0.738478,-0.228456,0.667732,0.069791,-0.029932,0.054902,NORMAL,3581631
4,0.101741,0.777568,-0.066730,0.030696,-0.003665,0.054902,NORMAL,3581631
...,...,...,...,...,...,...,...,...
3079,-0.713858,-0.652975,-0.164015,-0.147829,-1.309466,0.517250,SLOW,820706
3080,1.514261,0.330070,1.020714,1.321302,1.707598,-0.674548,SLOW,820707
3081,1.280216,-1.735172,-2.332695,0.583376,0.690507,-0.468075,SLOW,820707
3082,0.912313,0.583314,-0.965622,0.235794,0.512745,0.406073,SLOW,820708


In [3]:
data.duplicated().sum()

0

In [4]:
data.Class=data.Class.replace(['AGGRESSIVE', 'NORMAL', 'SLOW'],[1,2,3])


In [5]:
from sklearn.model_selection import train_test_split
x = data.drop('Class',axis = 1)
y = data['Class']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y)

In [6]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
x_train ,y_train = smote.fit_resample(x_train,y_train)

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

rf = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30], 
    'min_samples_split': [2, 5, 10], 
    'min_samples_leaf': [1, 2, 4], 
    'bootstrap': [True, False]  
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           cv=3, scoring='accuracy', verbose=2, n_jobs=-1)

grid_search.fit(x_train, y_train)

print("Best parameters found: ", grid_search.best_params_)

best_rf = grid_search.best_estimator_

y_pred = best_rf.predict(x_test)
print(f'The accuracy in train: {accuracy_score(y_train, best_rf.predict(x_train))}')
print(f'The accuracy in test: {accuracy_score(y_test, y_pred)}')
print(f'The confusion matrix: \n{confusion_matrix(y_test, y_pred)}')
print(f'The classification report: \n{classification_report(y_test, y_pred)}')


Fitting 3 folds for each of 216 candidates, totalling 648 fits
Best parameters found:  {'bootstrap': False, 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
The accuracy in train: 1.0
The accuracy in test: 0.9940564635958395
The confusion matrix: 
[[385   0   0]
 [  3 437   0]
 [  0   5 516]]
The classification report: 
              precision    recall  f1-score   support

           1       0.99      1.00      1.00       385
           2       0.99      0.99      0.99       440
           3       1.00      0.99      1.00       521

    accuracy                           0.99      1346
   macro avg       0.99      0.99      0.99      1346
weighted avg       0.99      0.99      0.99      1346



In [9]:
from joblib import dump
dump(best_rf, 'ML Model/model.pkl')


['D:\\my Machine learning models\\driver behavior\\new data\\model.pkl']